In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly as py
import pandas as pd
from plotly import tools
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dash import Dash

###################################################################################################
app = Dash(__name__)
app.config['suppress_callback_exceptions'] = True
df = pd.read_csv("Superstore.csv")
df = df.drop_duplicates()
df.duplicated().sum()
df = df.drop(df[df.Sales < 100].index)
df = df.drop(df[df.Profit == -6599.9780].index)
new_df = df.groupby(['Region'])['Sales'].mean()
new_df = pd.DataFrame(new_df)
new_df['Region'] = new_df.index

filter_df = pd.DataFrame({"categorytype":["Category","SubCategory"],"charttype":["Sales","Profit"]})


def categorise(row):
    if row['Discount'] > 0:
        return 'Yes'
    return 'No'
df['Discount_Status'] = df.apply(lambda row: categorise(row), axis=1)
dataset = df.groupby(['Category', 'Sub-Category','Discount_Status']).sum(['Sales']).reset_index()
category_list = list(dataset["Category"].unique())
category_list.append("All")  
####################################################################################################
app.layout = html.Div([

    html.Br(),
    html.Br(),
    
    dcc.RadioItems(
        id='type-radio',
        options=[{'label': k, 'value': k} for k in ['Sales', 'Profit']],
        value='Sales',
    ),
       html.Br(),
    html.Br(),
        dcc.Dropdown(
        id='Category-dropdown',
      options=[{'label': k, 'value': k} for k in category_list],
        value='Furniture',
            ),

    html.Hr(),
    html.Br(),
    html.Br(),
    html.Div(id='display-selected-values'),
    dcc.Graph(id="mkt_graph")
])


# Make a figure based on the selections
@app.callback( # Columns 2m_temp_prod, or....
    Output('mkt_graph', 'figure'),
    [
     Input('Category-dropdown', 'value'),
     Input('type-radio', 'value'),])
     
     
def make_graph(levels,type_graph):
    
    group_by_profit = dataset.groupby(["Sub-Category","Category","Discount_Status"]).sum().reset_index()
    title_all =  "All Categories"
    
    if levels != "All":
        title_all = levels
        group_by_profit = group_by_profit[group_by_profit["Category"]==levels]
    if type_graph!="Profit":
        fig = px.bar(group_by_profit, x="Sub-Category", y="Sales", color="Discount_Status", barmode='group', height=1000,color_discrete_sequence=["Orange","Green"])
        
        fig.update_layout(
            title={'text': "Sales by "+str(title_all),'y':1,'x':0.5,'xanchor': 'center','yanchor': 'top'},
            font_family="Calibri",
            font_size=20,
            font_color="black",
            title_font_family="Times New Roman",
            title_font_color="red",
            legend_title_font_color="black",
            legend_title_font_size=20,
            legend_title = "Discount Applied?",
        )
        fig.update_xaxes()
    
        
        
        
        
    else:
        
        fig = px.bar(group_by_profit, x="Sub-Category", y="Profit", color="Discount_Status", barmode='group', height=1000 ,color_discrete_sequence=["Orange","Green"] )
        
        fig.update_layout(
            title={'text': "Profit/Loss by "+str(title_all),'y':1,'x':0.5,'xanchor': 'center','yanchor': 'top'},
            font_family="Calibri",
            font_size=20,
            font_color="black",
            title_font_family="Times New Roman",
            title_font_color="red",
            legend_title_font_color="black",
            legend_title_font_size=20,
            legend_title = "Discount Applied?",
        )
        fig.update_xaxes(title={'text':str(levels)})
        #fig.update_yaxes(ticklabelposition="inside top", title=None)
    fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="reversed",
        title_font_family="Times New Roman",
        font=dict(
            family="Courier",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    )
)
 
    
    return fig


In [ ]:
print("loaded all , now running the server")
#if __name__ == '__main__':
app.run_server(port=8052,debug=False)